# Create Heat Map for significant Spliceosome genes

This notebook looks at the significant genes in at least one cancer in the DNA Replication pathway. Pancancer heat maps are created with circle size showing significance and color showing differences in median.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats
import gseapy as gp
import re
import sys 

import cptac
import cptac.utils as u

import plot_utils as p

# Step 1: Run GSEA for significant genes in at least 1 cancer

First read in sig_pval_all_proteins.csv into a df. This csv file contains only genes with a significant p-value in at least one cancer. Then run GSEA using a list of genes from the df.

In [2]:
root = R'~\Github\WhenMutationsDontMatter\PTEN\Step_3_trans_effect\csv'
sig_df = pd.read_csv(root+R'\sig_pval_heatmap.csv')

prot_list = list(sig_df.Proteomics) # list of genes with a sig pval in >= 1 cancer
prot_enr = gp.enrichr(gene_list = prot_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [3]:
prot_enr.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Spliceosome Homo sapiens hsa03040,84/134,3.960162e-41,1.160328e-38,0,0,4.801729,446.703392,RBM25;EIF4A3;DDX42;HNRNPU;PRPF19;USP39;EFTUD2;...,KEGG_2016
1,RNA transport Homo sapiens hsa03013,76/172,6.887335e-24,1.008995e-21,0,0,3.384607,180.509081,CYFIP2;NUP107;NUP188;RPP30;EIF4A3;PHAX;PNN;XPO...,KEGG_2016
2,DNA replication Homo sapiens hsa03030,22/36,1.947423e-11,1.901983e-09,0,0,4.681050,115.443730,RFC5;FEN1;RFC3;PCNA;RFC4;MCM7;RFC1;RFC2;PRIM1;...,KEGG_2016
3,Mismatch repair Homo sapiens hsa03430,16/23,6.740173e-10,4.937177e-08,0,0,5.328626,112.528681,RFC5;RFC3;PCNA;RFC4;RFC1;RFC2;RPA1;MLH1;POLD3;...,KEGG_2016
4,mRNA surveillance pathway Homo sapiens hsa03015,34/91,3.983668e-09,2.334429e-07,0,0,2.861941,55.352977,SMG1;RNMT;EIF4A3;MSI1;PPP2R2A;MSI2;SMG7;SMG6;P...,KEGG_2016


# Step 2: Get the list of significant genes 

In [4]:
index = 2
trans = prot_enr.res2d.Genes[index]
genes = trans.split(';')
print(prot_enr.res2d.Term[index])
print('total genes:',len(genes))

DNA replication Homo sapiens hsa03030
total genes: 22


# Step 3: Create HeatMap

Slice out genes from the DNA Replication pathway from the df with genes sig in > 1 cancer.

In [5]:
# sig > 1 cancer
bool_df = sig_df.Proteomics.isin(genes)
plot_df = sig_df[bool_df]
len(plot_df.Proteomics.unique())

22

In [6]:
p.plotCircleHeatMap(plot_df, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width= 800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

# Step 4: Create a HeatMap with both +/- differences in median

 Read in the pos_neg_df.csv to create a df with only genes that have a pos and neg difference in median in different cancers. Slice out the genes that have a pos and neg difference in median in the pathway using the list of genes with a significant p-value in the pathway. 

In [7]:
pos_neg_df = pd.read_csv(root+R'\pos_neg_df.csv')

In [8]:
get = pos_neg_df.Proteomics.isin(genes) # bool df where True has both pos and neg
genes_pn = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn.Proteomics.unique()

array(['MCM6', 'MCM4', 'RFC5', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2',
       'RFC4', 'SSBP1', 'POLA2'], dtype=object)

In [9]:
p.plotCircleHeatMap(genes_pn, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Sig in multiple cancers

In [10]:
mult_sig_df = pd.read_csv(root+R'\mult_sig_pval_heatmap.csv')

mult_sig_list = list(mult_sig_df.Proteomics.unique()) # list of genes with a sig pval in > 1 cancer
enr2 = gp.enrichr(gene_list = mult_sig_list, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

In [11]:
enr2.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,Ribosome biogenesis in eukaryotes Homo sapiens...,13/89,1.180367e-09,3.458475e-07,0,0,9.393403,193.104322,UTP6;IMP3;WDR3;HEATR1;NAT10;WDR75;IMP4;PWP2;UT...,KEGG_2016
1,DNA replication Homo sapiens hsa03030,9/36,3.083251e-09,4.516962e-07,0,0,16.077170,315.068832,RFC3;PCNA;RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM2,KEGG_2016
2,B cell receptor signaling pathway Homo sapiens...,11/73,1.626924e-08,1.588963e-06,0,0,9.690349,173.786622,PPP3CB;SYK;CHUK;PIK3CA;PRKCB;INPP5D;BLNK;RAC2;...,KEGG_2016
3,Fc gamma R-mediated phagocytosis Homo sapiens ...,12/93,2.188722e-08,1.603239e-06,0,0,8.297894,146.352976,PAK1;PTPRC;SYK;PIK3CA;MYO10;PRKCB;INPP5D;RAC2;...,KEGG_2016
4,RNA transport Homo sapiens hsa03013,15/172,8.324097e-08,4.877921e-06,0,0,5.608315,91.424098,RANBP2;NUP210;NUP155;NCBP1;NUP133;THOC3;THOC2;...,KEGG_2016


In [12]:
index2 = 1
trans2 = enr2.res2d.Genes[index2]
genes_mult = trans2.split(';')
print(enr2.res2d.Term[index2])
print('total genes:',len(genes_mult))

DNA replication Homo sapiens hsa03030
total genes: 9


In [13]:
bool_df = mult_sig_df.Proteomics.isin(genes_mult)
plot_df2 = mult_sig_df[bool_df]
plot_df2.Proteomics.unique()

array(['MCM4', 'PCNA', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC3', 'RFC2',
       'RFC4'], dtype=object)

In [14]:
p.plotCircleHeatMap(plot_df2, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=800)

C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["size2"] = df[circle_var].apply(lambda x: -1*(np.log(x)))
C:\Users\brittany henderson\Anaconda3\envs\Research\lib\site-packages\plot_utils\__init__.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['size'] = (df["size2"])*3


Loading BokehJS ...

In [15]:
get = pos_neg_df.Proteomics.isin(genes_mult) # bool df where True has both pos and neg
genes_pn_mult = pos_neg_df[get] # Keep only genes with pos and neg
genes_pn_mult.Proteomics.unique()

array(['MCM4', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2', 'RFC4'],
      dtype=object)

In [16]:
p.plotCircleHeatMap(genes_pn_mult, circle_var = 'P_Value', color_var='Medians', x_axis= 'Proteomics', y_axis = 'Cancer',
                    plot_height=400, plot_width=600)

Loading BokehJS ...

# Check other pathways the genes (+/- and sig in mult. cancers) are part of

In [17]:
gnm = list(genes_pn_mult.Proteomics.unique())
print(gnm)
enr3 = gp.enrichr(gene_list = gnm, description='Tumor_partition', gene_sets='KEGG_2016', 
                       outdir='/Enrichr')

['MCM4', 'MCM5', 'MCM2', 'MCM7', 'MCM3', 'RFC2', 'RFC4']


In [18]:
enr3.res2d.head()

,Term,Overlap,P-value,Adjusted P-value,Old P-value,Old Adjusted P-value,Odds Ratio,Combined Score,Genes,Gene_set
0,DNA replication Homo sapiens hsa03030,7/36,3.287196e-20,9.631485e-18,0,0,555.555556,24923.148241,RFC4;MCM7;RFC2;MCM3;MCM4;MCM5;MCM2,KEGG_2016
1,Cell cycle Homo sapiens hsa04110,5/124,1.755969e-10,2.572494e-08,0,0,115.207373,2587.883676,MCM7;MCM3;MCM4;MCM5;MCM2,KEGG_2016
2,Mismatch repair Homo sapiens hsa03430,2/23,2.647207e-05,2.585439e-03,0,0,248.447205,2618.489528,RFC4;RFC2,KEGG_2016
3,Nucleotide excision repair Homo sapiens hsa03420,2/47,1.126576e-04,8.252169e-03,0,0,121.580547,1105.307899,RFC4;RFC2,KEGG_2016


In [19]:
# Look at p-values 
certain_gene = 'PCNA'
all_df = pd.read_csv(root+R'\all_heatmap.csv')
gene_df = all_df.loc[all_df['Proteomics'] == certain_gene]
gene_df

,Proteomics,P_Value,Medians,Cancer
131,PCNA,0.000019,0.631486,Gbm
11317,PCNA,0.000844,0.420740,Hnscc
22915,PCNA,0.009321,0.501850,Luad
40121,PCNA,0.628051,-0.070550,Lscc
46909,PCNA,0.139751,0.361150,Brca
59111,PCNA,0.413633,0.028375,Ov
66382,PCNA,0.012305,-0.235000,En
76275,PCNA,0.027668,-0.142000,Colon
